In [1]:
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix

In [2]:
ratings = pd.read_csv("data/rating.csv")

In [4]:
# Categorical codes for users and movies
user_ids  = ratings['userId'].astype('category').cat.codes.to_numpy(dtype=np.int64, copy=False)
movie_ids = ratings['movieId'].astype('category').cat.codes.to_numpy(dtype=np.int64, copy=False)

n_users  = int(user_ids.max()) + 1
n_movies = int(movie_ids.max()) + 1
n_ratings = len(ratings)

# We will put a 1 for the user, and a 1 for the movie (shifted by n_users)
# So A has shape (n_ratings, n_users + n_movies)
row_idx = np.repeat(np.arange(n_ratings, dtype=np.int64), 2)  # each rating contributes 2 entries
col_idx = np.empty(2 * n_ratings, dtype=np.int64)

# First entry: user column
col_idx[0::2] = user_ids
# Second entry: movie column, shifted by n_users
col_idx[1::2] = n_users + movie_ids

data = np.ones(2 * n_ratings, dtype=np.float64)

A_design = csr_matrix(
    (data, (row_idx, col_idx)),
    shape=(n_ratings, n_users + n_movies)
)

b = ratings['rating'].astype(float).to_numpy()

In [6]:
from scipy.sparse.linalg import LinearOperator


def build_rhs(A, b):
    """Compute f = A^T b as a dense 1D NumPy array."""
    return (A.T @ b).astype(np.float64)


def build_normal_system(A, lam):
    """Return a matrix-free LinearOperator for M x = (A^T A + lam**2 I) x.

    Parameters
    ----------
    A : scipy.sparse matrix
        Design matrix.
    lam : float
        Regularization parameter (lambda).
    """
    n = A.shape[1]
    lam2 = float(lam) ** 2

    def matvec(x):
        # x is a 1D NumPy array; compute A^T (A x) + lam^2 x without forming A^T A.
        y = A @ x
        z = A.T @ y
        if lam2 != 0.0:
            z = z + lam2 * x
        return z

    return LinearOperator(shape=(n, n), matvec=matvec, dtype=np.float64)

